# To find the best location for a Tool shop in toronto

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))


In [3]:
data = pd.DataFrame(df[0])
data = data.rename(columns={'Postal Code':'PostalCode', 'Borough':'Bourough','Neighbourhood':'Neighborhood'})
data=data[data['Bourough']!='Not assigned']
data

,PostalCode,Bourough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df2=data.groupby(['PostalCode', 'Bourough']).apply(lambda group: ', '.join(group['Neighborhood']))
df2=df2.to_frame().reset_index()
df2=df2.rename(columns={0:'Neighborhood'})
df2.loc[df2.Neighborhood == 'Not assigned', 'Neighborhood' ] = df2.Bourough
df2.head()

,PostalCode,Bourough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
!wget -O to_geo_space.csv http://cocl.us/Geospatial_data


--2021-01-05 13:51:33--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2021-01-05 13:51:33--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-05 13:51:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-05 13:51:35--  https://ibm.box.com/public/static/9afzr8

In [6]:
gf = pd.read_csv('to_geo_space.csv')
gf = gf.rename(columns={'Postal Code':'PostalCode'})
gf_new = pd.merge(df2, gf, on='PostalCode', how='inner')
gf_new.head()

,PostalCode,Bourough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Utilising Foursquare

In [7]:
CLIENT_ID='GCWC4PQ5TOBTK5AF2K3YE1HHS11TX11V1EJ2ELH2FCGHKVOD'
CLIENT_SECRET='5MUKMHTBJMV30UAI43O5SWCI0WQ3WPEZYFGBPBQ24HCF55EW'
VERSION='20210102'

In [8]:
from pandas.io.json import json_normalize
LIMIT=200
RADIUS=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
      
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
TO_venues = getNearbyVenues(names=gf_new['Neighborhood'],
                                   latitudes=gf_new['Latitude'],
                                   longitudes=gf_new['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [10]:
TO_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",24,24,24,24,24,24
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Willowdale East",35,35,35,35,35,35
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,3,3,3,3,3,3


In [11]:
print('Unique Venue Categories:')
list(TO_venues['Venue Category'].unique())

Unique Venue Categories:


['Fast Food Restaurant',
 'Bar',
 'Bank',
 'Electronics Store',
 'Restaurant',
 'Mexican Restaurant',
 'Rental Car Location',
 'Medical Center',
 'Intersection',
 'Breakfast Spot',
 'Coffee Shop',
 'Korean BBQ Restaurant',
 'Caribbean Restaurant',
 'Hakka Restaurant',
 'Thai Restaurant',
 'Athletics & Sports',
 'Gas Station',
 'Bakery',
 'Fried Chicken Joint',
 'Playground',
 'Department Store',
 'Discount Store',
 'Hobby Shop',
 'Ice Cream Shop',
 'Bus Line',
 'Park',
 'Soccer Field',
 'Motel',
 'American Restaurant',
 'Café',
 'General Entertainment',
 'Skating Rink',
 'College Stadium',
 'Chinese Restaurant',
 'Indian Restaurant',
 'Vietnamese Restaurant',
 'Pet Store',
 'Light Rail Station',
 'Sandwich Place',
 'Middle Eastern Restaurant',
 'Shopping Mall',
 'Auto Garage',
 'Latin American Restaurant',
 'Lounge',
 'Italian Restaurant',
 'Noodle House',
 'Pizza Place',
 'Pharmacy',
 'Convenience Store',
 'Grocery Store',
 'Furniture / Home Store',
 'Golf Course',
 'Pool',
 'Mediterr

In [12]:
tool_list=['Auto Garage','Bike Shop','Furniture / Home Store','Harbor / Marina','Airport Service','Construction & Landscaping','Auto Workshop','Mobile Phone Shop',
'Stadium','Climbing Gym','Plane','College Gym','Building','College Rec Center','Sculpture Garden','Indoor Play Area','Business Service','Stationery Store','Home Service',
'Gym','Arts & Crafts Store','Theater','Motel','Bus Station','Metro Station','Department Store','Gas Station']
tool_pd = pd.DataFrame(tool_list)
tool_pd = tool_pd.rename(columns={0:'Venue Category'})
TO_new = pd.merge(TO_venues, tool_pd, on='Venue Category', how='right')

In [13]:
TO_new.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",1,1,1,1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1,1,1,1
"Brockton, Parkdale Village, Exhibition Place",4,4,4,4,4,4
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",1,1,1,1,1,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",4,4,4,4,4,4
Canada Post Gateway Processing Centre,2,2,2,2,2,2
Cedarbrae,1,1,1,1,1,1
Central Bay Street,3,3,3,3,3,3
Church and Wellesley,2,2,2,2,2,2


## One hot encoding

In [14]:
TO_new_onehot = pd.get_dummies(TO_new[['Venue Category']], prefix="", prefix_sep="")

TO_new_onehot['Neighborhood'] = TO_new['Neighborhood'] 

fixed_columns = [TO_new_onehot.columns[-1]] + list(TO_new_onehot.columns[:-1])
TO_new_onehot = TO_new_onehot[fixed_columns]

TO_new_onehot.head()

,Neighborhood,Airport Service,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,Building,Bus Station,Business Service,Climbing Gym,...,Home Service,Indoor Play Area,Metro Station,Mobile Phone Shop,Motel,Plane,Sculpture Garden,Stadium,Stationery Store,Theater
0,Cedarbrae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Clarks Corners, Tam O'Shanter, Sullivan",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Thorncliffe Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
TO_grouped = TO_new_onehot.groupby('Neighborhood').mean().reset_index()
TO_grouped.shape
TO_grouped.head()



,Neighborhood,Airport Service,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,Building,Bus Station,Business Service,Climbing Gym,...,Home Service,Indoor Play Area,Metro Station,Mobile Phone Shop,Motel,Plane,Sculpture Garden,Stadium,Stationery Store,Theater
0,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.25,0.0,0.0
3,"Business reply mail Processing Centre, South C...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
4,"CN Tower, King and Spadina, Railway Lands, Har...",0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.00,0.0,0.0


#  Clustering Using K Means algorithm

In [16]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np


TO_grouped_clustering = TO_grouped.drop('Neighborhood', 1)
kclusters = np.arange(2,10)
results = {}
for size in kclusters:
    model = KMeans(n_clusters = size).fit(TO_grouped_clustering)
    predictions = model.predict(TO_grouped_clustering)
    results[size] = silhouette_score(TO_grouped_clustering, predictions)

best_size = max(results, key=results.get)
best_size


8

In [17]:
from sklearn.cluster import KMeans
kclusters =best_size 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TO_grouped_clustering)
kmeans.labels_[0:10]



array([5, 2, 3, 3, 3, 5, 2, 3, 0, 2], dtype=int32)

In [18]:
TO_labels = pd.merge(gf_new,TO_grouped, on='Neighborhood', how='right')
TO_labels.head()

,PostalCode,Bourough,Neighborhood,Latitude,Longitude,Airport Service,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,...,Home Service,Indoor Play Area,Metro Station,Mobile Phone Shop,Motel,Plane,Sculpture Garden,Stadium,Stationery Store,Theater
0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = TO_grouped['Neighborhood']

for ind in np.arange(TO_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TO_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(48, 11)

In [20]:
TO_labels = pd.merge(gf_new,TO_grouped, on='Neighborhood', how='right')
TO_labels = TO_labels.drop(columns=['Auto Garage','Bike Shop','Furniture / Home Store','Harbor / Marina','Airport Service','Construction & Landscaping','Auto Workshop','Mobile Phone Shop',
'Stadium','Climbing Gym','Plane','College Gym','Building','College Rec Center','Sculpture Garden','Indoor Play Area','Business Service','Stationery Store','Home Service',
'Gym','Arts & Crafts Store','Theater','Motel','Bus Station','Metro Station','Department Store','Gas Station'])
TO_labels.head()



,PostalCode,Bourough,Neighborhood,Latitude,Longitude
0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
1,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
2,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
3,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
4,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302


In [22]:
TO_merged = TO_labels
TO_merged=TO_merged.drop([45])


TO_merged['Cluster Labels'] = kmeans.labels_
TO_merged = TO_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
TO_merged.head()






,PostalCode,Bourough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,5,Gas Station,Theater,Department Store,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,Building,Bus Station,Business Service
1,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,2,Department Store,Theater,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,Building,Bus Station,Business Service,Climbing Gym
2,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3,Motel,Theater,Department Store,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,Building,Bus Station,Business Service
3,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,3,Auto Garage,Theater,Department Store,Arts & Crafts Store,Auto Workshop,Bike Shop,Building,Bus Station,Business Service,Climbing Gym
4,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302,3,Gas Station,Theater,Department Store,Arts & Crafts Store,Auto Garage,Auto Workshop,Bike Shop,Building,Bus Station,Business Service


In [23]:
TO_merged_new1 = TO_merged.loc[TO_merged['Cluster Labels'] == 0, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new1.shape

(8, 13)

In [24]:

TO_merged_new2 = TO_merged.loc[TO_merged['Cluster Labels'] == 1, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [25]:

TO_merged_new3 = TO_merged.loc[TO_merged['Cluster Labels'] == 2, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [26]:

TO_merged_new5 = TO_merged.loc[TO_merged['Cluster Labels'] == 3, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [27]:

TO_merged_new10 = TO_merged.loc[TO_merged['Cluster Labels'] == 4, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [28]:
TO_merged_new10 = TO_merged.loc[TO_merged['Cluster Labels'] == 5, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [29]:
TO_merged_new10 = TO_merged.loc[TO_merged['Cluster Labels'] == 6, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [30]:
TO_merged_new10 = TO_merged.loc[TO_merged['Cluster Labels'] == 7, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [31]:
TO_merged_new10 = TO_merged.loc[TO_merged['Cluster Labels'] == 8, TO_merged.columns[[3, 4] + list(range(5, TO_merged.shape[1]))]]
TO_merged_new2.shape

(5, 13)

In [32]:
Cluster_0_coorid = TO_merged_new1[['Latitude', 'Longitude']]
Cluster_0_coorid = list(Cluster_0_coorid.values) 
lat = []
long = []



for l in Cluster_0_coorid:
  lat.append(l[0])
  long.append(l[1])



Blatitude = sum(lat)/len(lat)
Blongitude = sum(long)/len(long)
print(Blatitude)
print(Blongitude)

43.6677177375
-79.42570724999999


In [33]:
url2="https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            Blatitude, 
            Blongitude, 
            RADIUS, 
            LIMIT)
results = requests.get(url2).json()
results

{'meta': {'code': 200, 'requestId': '5ff46f97de8ca21350414607'},
 'response': {'venues': [{'id': '540c8301498e51456f71ae7e',
    'name': 'Contra Cafe',
    'location': {'address': '1028 Shaw St',
     'crossStreet': 'Yarmouth',
     'lat': 43.669107144072974,
     'lng': -79.42610471852032,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.669107144072974,
       'lng': -79.42610471852032}],
     'distance': 157,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['1028 Shaw St (Yarmouth)', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1609854871',
    'hasPerk': False},
   {'id': '4be1f594ae55a593f5275c62',
    'name': 'Essex Public School',
    'location'

### Best location for Tool Shop

In [36]:
popstring = gf_new[gf_new['PostalCode'].str.contains('M6G')]
def str_join(*args):
    return ''.join(map(str, args))
popstring_new = str_join('The Best Neighbourhood to locate a Tools Store is in: ', popstring['Neighborhood'].values,  ' in ' ,  popstring['Bourough'].values)
print(popstring_new)

The Best Neighbourhood to locate a Tools Store is in: ['Christie'] in ['Downtown Toronto']


In [37]:
!pip install folium 


     |████████████████████████████████| 93 kB 3.7 MB/s  eta 0:00:01


In [41]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude=43.653963
longitude=-79.387207 #toronto coordinates

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(TO_merged['Latitude'], TO_merged['Longitude'], TO_merged['Neighborhood'], TO_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
folium.CircleMarker([Blatitude, Blongitude],
                    radius=50,
                    popup='Toronto',
                    color='red',
                    ).add_to(map_clusters)

map_clusters.add_child(folium.ClickForMarker(popup=popstring_new))
map_clusters.save('map_clusters.png')       
map_clusters
